In [1]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = '''mseRGXB89UTFVkWJhTRTzzZ9Ujj4ZPbGK5'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount * 100000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = total - int(fee*100000000) - output_amount
h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
01000000013fdfef60ecd21c5e667cfe30fcb890a116688ca51ac3880f91008dd141ddcdb2080000006b483045022100b0453c379054fe909ce09d6a37eba3b8fc1fc4b7dcbe34e6a21125a513189ab402200ccefbb93951f881c93b195ae5f0d93c14aa1eda9680274bc0169f2089f778c20121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffff0280969800000000001976a914850af0029eb376691c3eef244c25eceb4e50c50388ace19c5a81000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac00000000


In [2]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'

secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = 'mseRGXB89UTFVkWJhTRTzzZ9Ujj4ZPbGK5'
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))

# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = randint(5000000, 20000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(target_satoshis, script_pubkey))

change_satoshis = total - int((fee)*100000000) - output_amount

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

0100000001adf63c0ff706522946f4b350bfc62fdf880cd4b80366e9aa8faea23a9d4b020c010000006b483045022100dfef0f6abe354efa4fac6d6a05be231518e742278f1a213947f80a70883a0f8502202c16dd85af3c0cb7ef1386f21e8ae7c2f6f77600f6ceb3d1b83fae4a5ddb88bd0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffff025494a100000000001976a914850af0029eb376691c3eef244c25eceb4e50c50388aced81b780000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac00000000
